In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from training.base import *
from training.ddp import *

Порежем обучающий датасет на два и зафиксируем срез

In [3]:
transposition = np.arange(60000)
np.random.seed(42)
np.random.shuffle(transposition)

In [20]:
DIM = 15910

**Эксперименты с маленькой моделькой:**

Запустим базовую модель

In [21]:
run_base(30, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='None', dim=DIM)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:18<00:00,  1.60it/s]


In [22]:
run_base(300, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='RandK', dim=DIM, alpha=0.1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [03:14<00:00,  1.54it/s]


In [23]:
run_base(300, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='MultiplicationPenalty', dim=DIM, alpha=0.1, penalty=0.2)

dim = 15910


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [03:21<00:00,  1.49it/s]


In [24]:
run_base(300, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='SubtractionPenalty', dim=DIM, alpha=0.1, penalty=0.5)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [03:19<00:00,  1.50it/s]


In [27]:
run_base(300, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='ExpSmoothing', dim=DIM, alpha=0.1, beta=0.3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [03:16<00:00,  1.52it/s]


In [26]:
run_base(300, 64, "./model_toy_batch_64_base", "./20_history_toy_batch_64_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False,
        compressor='BanLastM', dim=DIM, alpha=0.1, M=9)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [03:24<00:00,  1.47it/s]


In [ ]:
run_base(40, 128, "./model_toy_batch_128_base", "./history_toy_batch_128_base", transposition[:30000], use_fp16=False, use_mobilenetv3=False)

In [25]:
np.load('history_toy_batch_128_base/train_loss.npy')

array([1.1341083 , 0.47009417, 0.3746451 , 0.33545175, 0.31241184,
       0.29676858, 0.2851761 , 0.27600548, 0.2684637 , 0.26200932,
       0.25644776, 0.25155184, 0.24718201, 0.24315995, 0.23958501,
       0.2363034 , 0.23328514, 0.2305561 , 0.22798112, 0.22556111,
       0.22328307, 0.22117229, 0.21913777, 0.21717247, 0.21527846,
       0.2134638 , 0.21167774, 0.20999943, 0.20839247, 0.20675305,
       0.20524155, 0.20374237, 0.20223458, 0.2007726 , 0.19937965,
       0.19805367, 0.19671021, 0.19548531, 0.19422758, 0.19303165],
      dtype=float32)

In [27]:
run_base(40, 256, "./model_toy_batch_256_base_v2", "./history_toy_batch_256_base_v2", transposition[:30000], use_fp16=False, use_mobilenetv3=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 194.08it/s]


In [29]:
np.load('history_toy_batch_256_base_v2/train_loss.npy')

array([1.4078696 , 0.6367326 , 0.45630673, 0.3867037 , 0.3501284 ,
       0.32756433, 0.31189698, 0.2999482 , 0.29038727, 0.28241673,
       0.27552745, 0.26947498, 0.2641156 , 0.25930947, 0.25495723,
       0.25094423, 0.24725118, 0.24382022, 0.2406093 , 0.23755872,
       0.23471448, 0.23204313, 0.22952731, 0.22711957, 0.224846  ,
       0.22271408, 0.22067332, 0.21874097, 0.2168822 , 0.21510099,
       0.2134045 , 0.21182209, 0.21028937, 0.20881246, 0.20738497,
       0.20600471, 0.20470223, 0.20346065, 0.20223546, 0.20108299],
      dtype=float32)

Теперь запустим с ddp

In [5]:
run(ddp_train, 2, 15, 64, "./model_toy_batch_64_ddp2", "./history_toy_batch_64_ddp2",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 235/235 [00:04<00:00, 55.46it/s] 


In [5]:
run(ddp_train, 2, 15, 128, "./model_toy_batch_128_ddp2", "./history_toy_batch_128_ddp2",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 118/118 [00:04<00:00, 28.69it/s]


In [4]:
run(ddp_train, 2, 15, 256, "./model_toy_batch_256_ddp2", "./history_toy_batch_256_ddp2",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 59/59 [00:04<00:00, 14.48it/s]


In [5]:
run(ddp_train, 4, 15, 64, "./model_toy_batch_64_ddp4", "./history_toy_batch_64_ddp4",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 118/118 [00:04<00:00, 24.62it/s]


In [5]:
run(ddp_train, 4, 15, 128, "./model_toy_batch_128_ddp4", "./history_toy_batch_128_ddp4",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 59/59 [00:04<00:00, 12.38it/s]


In [4]:
run(ddp_train, 4, 15, 256, "./model_toy_batch_256_ddp4", "./history_toy_batch_256_ddp4",  transposition[:30000], use_fp16=False, use_mobilenetv3=False)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 30/30 [00:04<00:00,  6.44it/s]


**Теперь экспы с большой моделькой**

Без ddp

In [11]:
run_base(15, 64, "./model_mobile_batch_64_base", "./history_mobile_batch_64_base", transposition[:30000], use_fp16=True, use_mobilenetv3=True)

100%|██████████| 469/469 [07:14<00:00,  1.08it/s]


In [9]:
run_base(15, 128, "./model_mobile_batch_128_base", "./history_mobile_batch_128_base", transposition[:30000], use_fp16=True, use_mobilenetv3=True)

100%|██████████| 235/235 [05:02<00:00,  1.29s/it]


In [7]:
run_base(15, 256, "./model_mobile_batch_256_base_v2", "./history_mobile_batch_256_base_v2", transposition[:30000], use_fp16=True, use_mobilenetv3=True)

/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 118/118 [03:09<00:00,  1.60s/it]


In [12]:
np.load("./history_mobile_batch_64_base/val_acc.npy")

array([0.8077, 0.9035, 0.9298, 0.9284, 0.9238, 0.91  , 0.9513, 0.9528,
       0.954 , 0.961 , 0.9617, 0.9603, 0.9596, 0.9629, 0.9658],
      dtype=float32)

In [10]:
np.load("./history_mobile_batch_128_base/val_acc.npy")

array([0.7785, 0.8738, 0.905 , 0.9228, 0.9297, 0.9294, 0.9335, 0.9385,
       0.9418, 0.9464, 0.9495, 0.943 , 0.9519, 0.9489, 0.9521],
      dtype=float32)

Теперь с ddp

In [5]:
run(ddp_train, 2, 15, 64, "./model_mobile_batch_64_ddp2", "./history_mobile_batch_64_ddp2",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 235/235 [05:35<00:00,  1.43s/it]

100%|██████████| 235/235 [05:38<00:00,  1.44s/it]

100%|██████████| 235/235 [05:38<00:00,  1.44s/it]


In [4]:
run(ddp_train, 2, 15, 128, "./model_mobile_batch_128_ddp2", "./history_mobile_batch_128_ddp2",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 118/118 [03:32<00:00,  1.80s/it]


In [4]:
run(ddp_train, 2, 15, 256, "./model_mobile_batch_256_ddp2", "./history_mobile_batch_256_ddp2",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 59/59 [02:11<00:00,  2.22s/it]

100%|██████████| 59/59 [02:12<00:00,  2.24s/it]

100%|██████████| 59/59 [02:13<00:00,  2.26s/it]


In [6]:
np.load("./history_mobile_batch_64_ddp2/val_acc.npy")

array([0.7163, 0.8668, 0.9051, 0.9261, 0.9373, 0.9452, 0.9457, 0.9524,
       0.9529, 0.9523, 0.9591, 0.9518, 0.96  , 0.9577, 0.963 ],
      dtype=float32)

In [5]:
np.load("./history_mobile_batch_128_ddp2/val_acc.npy")

array([0.6419, 0.8189, 0.8745, 0.9081, 0.9189, 0.9291, 0.9323, 0.9368,
       0.9376, 0.9431, 0.9461, 0.9477, 0.9478, 0.9502, 0.9539],
      dtype=float32)

In [5]:
np.load("./history_mobile_batch_256_ddp2/val_acc.npy")

array([0.5304, 0.7027, 0.7954, 0.8529, 0.8765, 0.8907, 0.8981, 0.9084,
       0.9136, 0.9159, 0.921 , 0.9197, 0.9226, 0.9244, 0.9291],
      dtype=float32)

In [6]:
run(ddp_train, 4, 15, 64, "./model_mobile_batch_64_ddp4", "./history_mobile_batch_64_ddp4",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/118 [00:00<?, ?it/s]/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not availa

In [4]:
run(ddp_train, 4, 15, 128, "./model_mobile_batch_128_ddp4", "./history_mobile_batch_128_ddp4",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/59 [00:00<?, ?it/s]/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not availab

In [4]:
run(ddp_train, 4, 15, 256, "./model_mobile_batch_256_ddp4", "./history_mobile_batch_256_ddp4",  transposition[:30000], use_fp16=True, use_mobilenetv3=True)

Running basic DDP example on rank 0.
Running basic DDP example on rank 1.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 2.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


Running basic DDP example on rank 3.


[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
[W ProcessGroupGloo.cpp:725] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/30 [00:00<?, ?it/s]/opt/homebrew/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not availab

In [7]:
np.load("./history_mobile_batch_64_ddp4/val_acc.npy")

array([0.6384, 0.8139, 0.8577, 0.9042, 0.9226, 0.9312, 0.9402, 0.942 ,
       0.9457, 0.948 , 0.9477, 0.951 , 0.9523, 0.9554, 0.9531],
      dtype=float32)

In [5]:
np.load("./history_mobile_batch_128_ddp4/val_acc.npy")

array([0.5685, 0.7257, 0.8171, 0.8649, 0.8894, 0.9009, 0.9161, 0.9168,
       0.9195, 0.923 , 0.929 , 0.9285, 0.9267, 0.9313, 0.9365],
      dtype=float32)

In [9]:
np.load("./history_toy_batch_256_base_v2/val_acc.npy")

array([0.8185, 0.8705, 0.8882, 0.9001, 0.9086, 0.9126, 0.9145, 0.9171,
       0.9189, 0.9201, 0.921 , 0.9218, 0.9221, 0.9228, 0.9242],
      dtype=float32)

Пример подсчета статистик для таблички

In [3]:
np.load("./history_toy_batch_64_base/val_acc.npy")[4::5]

array([0.8809, 0.9059, 0.9165], dtype=float32)

In [71]:
np.load("./history_mobile_batch_256_ddp4/time_epoch.npy").mean()

95.10962

In [70]:
np.load("./history_mobile_batch_256_ddp4/time_batch.npy").mean()

2.654641